# Fetch manifest for a Dataset

The script in this notebook retrieves a manifest for a given Dataset.

Fetching a manifest for a Dataset requires only the Collection id and the Dataset id; it does not require an API key/access token.

### Import dependencies

In [ ]:
import requests

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Enter the id of the Collection that contains the Dataset for which you want to fetch full metadata</font>

_The Collection id can be found by looking at the url path in the address bar 
when viewing your Collection in the CZ CELLxGENE Discover data portal: `/collections/{collection_id}`._

In [ ]:
collection_id = "01234567-89ab-cdef-0123-456789abcdef"

<font color='#bc00b0'>(Required) Enter the id of the Dataset</font>

_The Dataset id can be found by using the `/collections/{collection_id}` endpoint and filtering for the Dataset of interest OR by looking at the url path in the address when viewing your Dataset using the CZ CELLxGENE Explorer browser tool: `/e/{dataset_id}.cxg/`._

In [ ]:
dataset_id = "abcdef01-2345-6789-abcd-ef0123456789"

### Specify domain (and API url)

In [ ]:
domain_name = "cellxgene.cziscience.com"
site_url = f"https://{domain_name}"
api_url_base = f"https://api.{domain_name}"

### Formulate request and fetch a manifest for a dataset

In [ ]:
dataset_path = f"/curation/v1/collections/{collection_id}/datasets/{dataset_id}/manifest"
url = f"{api_url_base}{dataset_path}"
res = requests.get(url=url)
res.raise_for_status()
res_content = res.json()
print(res_content)

### Download Dataset Assets

The dataset manifest provides download URLs for anndata and atac fragment assets. For public collections, that means the most recently published version of a dataset. For private collections, that means the most recently successfully processed dataset version.

These download URLs are permalinks to download the assets for this particular version of a dataset. If this dataset is revised, you would need to fetch the dataset manifest again to get the latest dataset version asset download links.

In [ ]:
for asset_url in res_content.values():
    download_filename = asset_url.split("/")[-1]
    print(f"\nDownloading {download_filename}... ")
    with requests.get(asset_url, stream=True) as res:
        res.raise_for_status()
        filesize = int(res.headers["Content-Length"])
        with open(download_filename, "wb") as df:
            total_bytes_received = 0
            for chunk in res.iter_content(chunk_size=1024 * 1024):
                df.write(chunk)
                total_bytes_received += len(chunk)
                percent_of_total_upload = float("{:.1f}".format(total_bytes_received / filesize * 100))
                color = "\033[38;5;10m" if percent_of_total_upload == 100 else ""
                print(f"\033[1m{color}{percent_of_total_upload}% downloaded\033[0m\r", end="")
print("\n\nDone downloading assets")